-----------------

In [168]:
import subprocess


In [169]:
#OpenAI Whisper command line
# To use the subtitles, can use: vlc test.mp4 --sub-file test2.srt
subprocess.call(['vlc', 'data/test.mp4', '--sub-file', 'test2.srt'])
# convert mp3 to wav file
#%timeit subprocess.call(['whisper', 'data/test.mp3', '--model', 'tiny.en', '-o', 'data', '-f', 'txt'])

[00005561c7d38550] main libvlc: Ejecutar vlc con la interfaz predeterminada. Use «cvlc» para usar vlc sin interfaz.
[00007fe3ac004930] gl gl: Initialized libplacebo v4.192.1 (API v192)
libva info: VA-API version 1.16.0
libva error: vaGetDriverNameByIndex() failed with unknown libva error, driver_name = (null)
[00007fe3ac004930] glconv_vaapi_x11 gl error: vaInitialize: unknown libva error
libva info: VA-API version 1.16.0
libva info: Trying to open /usr/lib/x86_64-linux-gnu/dri/iHD_drv_video.so
libva info: Found init function __vaDriverInit_1_14
libva error: /usr/lib/x86_64-linux-gnu/dri/iHD_drv_video.so init failed
libva info: va_openDriver() returns 1
libva info: Trying to open /usr/lib/x86_64-linux-gnu/dri/i965_drv_video.so
libva info: Found init function __vaDriverInit_1_10
libva info: va_openDriver() returns 0
[00007fe3ac004930] gl gl: Initialized libplacebo v4.192.1 (API v192)
Failed to open VDPAU backend libvdpau_nvidia.so: no se puede abrir el archivo del objeto compartido: No e

0

________


--------

In [ ]:
# Splitting video

#Importing library and thir function
from pydub import AudioSegment
from pydub.silence import split_on_silence

#reading from audio mp3 file
sound = AudioSegment.from_mp3("/content/Audio/song_with_silence.mp3")

# spliting audio files
audio_chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-40 )

#loop is used to iterate over the output list
for i, chunk in enumerate(audio_chunks):
   output_file = "/content/Audio/output/chunk{0}.mp3".format(i)
   print("Exporting file", output_file)
   chunk.export(output_file, format="mp3")

# chunk files saved as Output

In [39]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

audio = AudioSegment.from_file("data/test.mp4", format="mp4")

chunks = split_on_silence(audio, 
                         keep_silence=30000, 
                         min_silence_len=200, 
                         silence_thresh=-30)

for i, chunk in enumerate(chunks):
    chunk.export(f"data/Chunks/output_{i}.mp3", format="mp3")


In [ ]:

#Transcription splitting video

from pydub import AudioSegment
from pydub.silence import split_on_silence
import whisper
import os
from datetime import timedelta
from threading import Thread

sound = AudioSegment.from_file("/home/roque/01. IronHack/00. Data Analytics/01. Course/01. Week 1 - Day 1/Recordings/GMT20220906-165929_Recording_1920x1080.mp4", format="mp4")
chunks = split_on_silence(
    sound,

    # split on silences longer than 1000ms (1 sec)
    min_silence_len=1000,

    # anything under -16 dBFS is considered silence
    silence_thresh=-30, 

    # keep 200 ms of leading/trailing silence
    keep_silence=200
)

# now recombine the chunks so that the parts are at least 30 sec long
target_length = 30 * 1000
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length,
        # we can start a new one
        output_chunks.append(chunk)

# now your have chunks that are bigger than 30 seconds (except, possibly the last one)

n_threads = 8
threads = []

for i, chunk in enumerate(output_chunks):
    name = f"data/Chunks/output_{i}.mp3"
    format="mp3"
    t = Thread(target=chunk.export,args=(name, format))
    t.start()
    threads.append(t)

for t in threads:
    t.join()


# Get a list of all the audio files in the "data/Chunks/" folder
audio_files = [f for f in sorted(os.listdir("data/Chunks/")) if f.endswith('.mp3')]

# Initialize an empty list to store the transcriptions
transcriptions = []
segments = []

model = whisper.load_model("base.en",device='cpu')
#model = stable_whisper.load_model('base')

# Loop over all the audio files in the "data/Chunks/" folderL_

for audio_file in audio_files:
    audio_file_path = os.path.join("data/Chunks/", audio_file)
    result = model.transcribe(audio_file_path)
    #transcription = str(result)
    transcriptions.append(result['text'])
    #print(len(result['segments']))
    #segments.append([res for res in result['segments']])
    [segments.append(s) for s in result['segments']]
    #segments.append(result['segments'])

    os.remove(audio_file_path)

with open("data/GMT20220906-165929_Recording_1920x1080.txt", "w+") as f:
    f.write(' '.join(transcriptions))

# subtitles
# TODO:Need to fix

#segments_final = repairTranscriptSegments([seg for segment in segments for seg in segment],40)
#segments_final = [s for seg in segments for s in seg]
#segments = result['segments']

"""ending = ''
id_end = 0
i = 0
segments_final = []
for segm in segments:         
    if (i > 0):        
        if (segm['id'] == i):
            i = segm['id']
        else:
            segm['id'] = i
            print(segments[i-1]['end'])
            segm['start'] = round(float(segm['start']+segments[i-1]['end']),2)
            segm['end'] = round(float(segm['end']+segments[i-1]['end']),2)
  
    segments_final.append(segm)
  
    i += 1      

for seg in segments_final:    
    startTime = str(0)+str(timedelta(seconds=int(seg['start'])))+',000'
    endTime = str(0)+str(timedelta(seconds=int(seg['end'])))+',000'
    text = seg['text']
    segmentId = seg['id']+1
    segmentFinal = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] == ' ' else text}\n\n"

    srtFilename = os.path.join(r"data", "test.srt")
    with open(srtFilename, 'a', encoding='utf-8') as srtFile:
        srtFile.write(segmentFinal)
"""    

# subtitles 
result_final = {}
result_final['text'] = ' '.join(transcriptions)
result_final['segments'] = segments
result_final['language'] = 'en'
stable_whisper.results_to_sentence_srt(result_final, "data/GMT20220906-165929_Recording_1920x1080.srt")


------

In [29]:
#summarization

import pandas as pd
from transformers import pipeline
import nltk

text = open('data/GMT20221029-085534_Recording_1920x1080.txt').read()

model = 'pszemraj/led-large-book-summary'

#summarizer = pipeline("summarization", model, truncation=True)
summarizer = pipeline("summarization", truncation=True)
mysummary = summarizer(text, min_length=100, max_length=250)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
print(mysummary[0]['summary_text'])
print('<<<<>>>>')
print(text)

In [28]:
def nest_sentences(document):

  nested = []
  sent = []
  length = 0
  for sentence in nltk.sent_tokenize(document):
    length += len(sentence)
    if length < 1024:
      sent.append(sentence)
    else:
      nested.append(sent)
      sent = []
      length = 0

  if sent:
    nested.append(sent)

  return nested

In [36]:
nested = nest_sentences(text)
len(nested)

87

In [46]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
import torch

BART_PATH = 'facebook/bart-large-cnn'

bart_model = BartForConditionalGeneration.from_pretrained(BART_PATH, output_past=True)
bart_tokenizer = BartTokenizer.from_pretrained(BART_PATH, output_past=True)

def generate_summary(nested_sentences):
  device = "cuda" if torch.cuda.is_available() else "cpu"  
  summaries = []
  for nested in nested_sentences:
    input_tokenized = bart_tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = bart_model.to(device).generate(input_tokenized,
                                      length_penalty=3.0,
                                      min_length=30,
                                      max_length=100)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [47]:
summ = generate_summary(nested)

KeyboardInterrupt: 

In [ ]:

nested_summ = nest_sentences(' '.join(summ))
nested_summ

In [ ]:
generate_summary(nested_summ)

In [8]:
#Connect to mysql
import mysql.connector

#Operating system
import os
import sys
from pathlib import Path
#import subprocess
from datetime import timedelta
from threading import Thread

#Pandas
import pandas as pd

In [9]:
def connection_ddbb():
    """Function to create the conection to the data base
    
    Keyword arguments:    
    Return: connection objet, secrets object
    """
       
    secrets={}
    #secrets_file = open('secrets.txt','r') #Had errors using this within a virtual environment
    secrets_file = os.fdopen(os.open('secrets.txt', os.O_RDONLY))
    for line in secrets_file:
        (key, val) = line.replace('\n','').split("|")
        secrets[key] = val    

    #Conection to mysql

    conn = mysql.connector.connect(user=secrets['user'],
                            password=secrets['pass'],
                            host=secrets['server'])
    
    return conn, secrets

In [10]:
connection, secrets = connection_ddbb()

In [11]:
import re

In [36]:
cursor = connection.cursor()
query = """SELECT transcription
                    FROM ironrep.transcriptions
                WHERE videoid = %s
                    AND languageid = %s"""
val = [int(45), 'en']
cursor.execute(query, val)    
subt_table = cursor.fetchall()
if (len(subt_table)>0):
    subt_df = pd.DataFrame(subt_table)
    subt_df.columns = [i[0] for i in cursor.description]
    transcription = subt_df['transcription'][0]




In [26]:
len(transcription)

from transformers import pipeline

In [55]:

#hub_model_id = "JulesBelveze/t5-small-headline-generator" #OK for headline 11sg
#hub_model_id = "ybagoury/flan-t5-base-tldr_news" #Interesting but maybe not 1m19sg
hub_model_id = "mrm8488/flan-t5-large-finetuned-openai-summarize_from_feedback" #OK Muy bueno 1m
summarizer = pipeline("summarization", model=hub_model_id)
summary = summarizer(' '.join(transcription.split(' ')[:250]), max_length=250)

In [52]:
#hub_model_id = "JulesBelveze/t5-small-headline-generator" #OK for headline
summary

[{'summary_text': "I'm recording the whole idea behind hypothesis testing. Let's talk about what goodness of fit is and how we can use it. Here's what we've learned"}]

In [54]:
#hub_model_id = "ybagoury/flan-t5-base-tldr_news" Interesting but maybe not
summary

[{'summary_text': "I'm going to talk about what goodness of fit is and how we can use it. So let's talk about the whole idea behind hypothesis testing. We have some hypothesis that we want to test, hence the name, which is something that can either be true or false. And the way we go about it is we test it. And we say, well, under one of these hypothesis, the one we call a null hypothesis, how likely is it to observe these sample that we actually have extracted from reality. And if what we observed from our sampling of reality, straight too far away from what our nul hypothesis would say, we say , okay, there is no evidence here to reject this this preconceived idea that we have against the world about the world "}]

In [56]:
#hub_model_id = "mrm8488/flan-t5-large-finetuned-openai-summarize_from_feedback" 
summary

[{'summary_text': "you take a sample of reality and you say, if it is consistent with your null hypothesis, then you say, okay, I'd rather believe that our null hypothesis was false, then to believe that I live in a world where I was so unlucky that I witnessed this sample that straced so far away from what seems to be possible."}]

In [58]:
summary[0]['summary_text']

"you take a sample of reality and you say, if it is consistent with your null hypothesis, then you say, okay, I'd rather believe that our null hypothesis was false, then to believe that I live in a world where I was so unlucky that I witnessed this sample that straced so far away from what seems to be possible."

' '.join(transcription.split(' ')[:250])